This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# function to generate url
def generate_url(stock, start_date, end_date):
    url = 'https://www.quandl.com/api/v3/datasets/FSE/%s?start_date=%s&end_date=%s&api_key=%s'%(stock, start_date, end_date, API_KEY)
    return url

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = generate_url(stock='AFX_X', start_date='2017-01-02', end_date='2017-01-03')
r = requests.get(url)

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-06-25T22:37:05.976Z',
  'newest_available_date': '2019-06-25',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-02',
  'end_date': '2017-01-03',
  'data': [['2017-01-03',
    35.9,
    35.93,
    35.34,
    35.48,
    None,
    70618.0,
    2515473.0,
    None,
    None,
    None],
   ['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
carl_zeiss_2017_url = generate_url(stock='AFX_X', start_date='2017-01-01', end_date='2017-12-31')
carl_zeiss_2017_response = requests.get(carl_zeiss_2017_url)

2. Convert the returned JSON object into a Python dictionary.

In [7]:
carl_zeiss_2017_json = carl_zeiss_2017_response.json()
type(carl_zeiss_2017_json)

dict

Extracting columns dictionary and data from original json

In [8]:
# Extracting data
data = carl_zeiss_2017_json['dataset']['data']

# Reversing data since original data is in latest to oldest order
data = list(reversed(data))

# Converting column_names from list to dictionary, so that its easy to fetch index of required column
columns = {column: index for index, column in enumerate(carl_zeiss_2017_json['dataset']['column_names'])}
columns

{'Date': 0,
 'Open': 1,
 'High': 2,
 'Low': 3,
 'Close': 4,
 'Change': 5,
 'Traded Volume': 6,
 'Turnover': 7,
 'Last Price of the Day': 8,
 'Daily Traded Units': 9,
 'Daily Turnover': 10}

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
open_index = columns['Open']

# Looping through all non-null open prices
daily_open_prices = [row[open_index] for row in data if type(row[open_index]) is float]

# Highest and Lowest opening prices
max(daily_open_prices), min(daily_open_prices)

(53.11, 34.0)

4. What was the largest change in any one day (based on High and Low price)?

In [10]:
high_index = columns['High']
low_index = columns['Low']
daily_price_change = [row[high_index] - row[low_index]  for row in data]

# Largest change in any one day
max(daily_price_change)

2.8100000000000023

5. What was the largest change between any two days (based on Closing Price)?

In [11]:
close_index = columns['Close']

# Absolute difference between closing price of a day with closing price of previous day
close_changes = [abs(row[close_index] - data[index-1][close_index]) for index, row in enumerate(data) if index>0]

# Largest change
max(close_changes)

2.559999999999995

6. What was the average daily trading volume during this year?

In [12]:
trading_volume_index = columns['Traded Volume']
trading_volume = [row[trading_volume_index] for row in data]

# Average of Traded Volume
sum(trading_volume)/len(trading_volume)

89124.33725490196

7. (Optional) What was the median trading volume during this year.

In [13]:
sorted_trading_volume = sorted(trading_volume)

# Taking the median of Traded Volume
median_index = len(sorted_trading_volume)//2 + 1
sorted_trading_volume[median_index]

76600.0